# Gun Violence in Philadelphia 2015 - 2020 - Exploratory Data Analysis

In this notebook, I have performed exploratory data analysis on this [Dataset](https://phl.carto.com/api/v2/sql?q=SELECT+*,+ST_Y(the_geom)+AS+lat,+ST_X(the_geom)+AS+lng+FROM+shootings&filename=shootings&format=csv&skipfields=cartodb_id) of gun violence incidents reported in Philadelphia.  Since 2020, gun violence has surged in Philadelphia.  In this notebook, I have performed a deep exploration of gun violence incidents reported in Philadelphia since 2015. 

As the first step, I imported the required libraries, loaded the dataset, and created some additional features.

In [78]:
import plotly.express as px
import plotly.offline as pyo 
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)
import pandas as pd
import numpy as np
import folium
import calendar

In [79]:
shootings_df = pd.read_csv("shootings.csv")
shootings_df['date_'] = pd.to_datetime(shootings_df['date_'])
shootings_df['year'] = shootings_df['date_'].dt.year
shootings_df['month'] = shootings_df['date_'].dt.month
shootings_df['monthday'] = shootings_df['date_'].dt.day
shootings_df['weekday'] = shootings_df['date_'].dt.weekday

## The dataset is comprised of 10,658 rows and 28 columns

In [80]:
shootings_df.shape

(10658, 28)

In [81]:
shootings_df['date_'].max()

Timestamp('2021-09-22 00:00:00')

In [82]:
shootings_df.isna().sum()

the_geom                203
the_geom_webmercator    203
objectid                  0
year                      0
dc_key                    0
code                     88
date_                     0
time                     88
race                     88
sex                       0
age                     151
wound                   108
officer_involved          0
offender_injured          0
offender_deceased         0
location                  0
latino                   88
point_x                 203
point_y                 203
dist                      2
inside                   88
outside                  88
fatal                    88
lat                     203
lng                     203
month                     0
monthday                  0
weekday                   0
dtype: int64

In [83]:
total_shootings_per_year = shootings_df.groupby('year')['objectid'].count()
year_labels = total_shootings_per_year.index.tolist()
total_shootings = total_shootings_per_year.values.tolist()

In [84]:
fig = px.bar(total_shootings_per_year, x=year_labels, y=total_shootings, 
            text=total_shootings, opacity=0.75)
fig.update_traces(texttemplate='%{text:,}', textposition='outside')
fig.update_layout(title_text='Total Shooting Incidents Per Year in Philadelphia', title_x=0.5)
fig.update_yaxes(visible=False)
fig.update_xaxes(title='', visible=True, showticklabels=True)
fig.show()

## Since 2017, shooting incidents have increased each year in Philadelphia.  In 2020, total shooting incidents increased by 51.69% over 2019.

In [85]:
total_shootings_per_year = total_shootings_per_year.reset_index()

In [86]:
total_shootings_per_year = total_shootings_per_year.rename(index=str, columns={'objectid': 'shooting incidents'})
format_dict = {'shooting incidents':'{0:,}'}
total_shootings_per_year['shooting incidents % change'] = total_shootings_per_year['shooting incidents'].pct_change()
format_dict = {'shooting incidents':'{0:,}', 'shooting incidents % change': '{:.2%}'}
total_shootings_per_year.style.format(format_dict).hide_index()

year,shooting incidents,shooting incidents % change
2015,"1,258",nan%
2016,"1,300",3.34%
2017,"1,235",-5.00%
2018,"1,441",16.68%
2019,"1,475",2.36%
2020,"2,253",52.75%
2021,"1,696",-24.72%


In [87]:
fatal_shootings_per_year = shootings_df.groupby('year')['fatal'].sum()
year_labels1 = fatal_shootings_per_year.index.tolist()
fatal_shootings = fatal_shootings_per_year.values.tolist()

In [88]:
fig = px.bar(fatal_shootings_per_year, x=year_labels1, y=fatal_shootings, 
            text=fatal_shootings, opacity=0.75)
fig.update_traces(texttemplate='%{text:,}', textposition='outside')
fig.update_layout(title_text='Gun Violence Deaths Per Year in Philadelphia', title_x=0.5)
fig.update_yaxes(visible=False)
fig.update_xaxes(title='', visible=True, showticklabels=True)
fig.show()

## 2020 was marked by a major increase in fatal gun shooting incidents in Philadelphia, representing a 36.18% increase over 2019 

In [89]:
month_df = shootings_df[shootings_df['year'].isin([2015, 2016, 2017, 2018, 2019, 2020])]
month_df1 = month_df.groupby(['year', 'month']).agg({'month' : 'count'}).rename(columns={'month': 'month_avg'}).reset_index()
grouped_month_avg = month_df1.groupby(['month']).agg({'month_avg': 'mean'})
grouped_month_x = grouped_month_avg.index.tolist()
grouped_month_y = grouped_month_avg.month_avg.tolist()
month_name = [calendar.month_abbr[i] for i in sorted(grouped_month_x)]


In [90]:
fig = px.bar(grouped_month_avg, x=month_name, y=grouped_month_y, 
            text=grouped_month_y, opacity=0.75)
fig.update_traces(texttemplate='%{text:.1f}', textposition='outside')
fig.update_layout(title_text='Average Number of Shooting Incidents Per Month in Philadelphia', title_x=0.5)
fig.update_yaxes(visible=False)
fig.update_xaxes(title='', visible=True, showticklabels=True)
fig.show()

## Since 2015, the months of August, July and October have averaged the highest number of shooting incidents.  The winter months of January, February and March have consistently averaged the lowest number of shooting incidents. 

In [91]:
weekday_df = shootings_df[shootings_df['year'].isin([2015, 2016, 2017, 2018, 2019, 2020])]
weekday_df1 = weekday_df.groupby(['year','weekday']).agg({'weekday': 'count'}).rename(columns={'weekday' : 'weekday_count'}).reset_index()

grouped_weekday_df = weekday_df1.groupby(['weekday']).agg({'weekday_count' : 'mean'}).round(1)

weekday_labels = grouped_weekday_df.index.tolist()
weekday_values = grouped_weekday_df.weekday_count.tolist()

weekmap = {0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
weekday_labels = [weekmap[x] for x in weekday_labels]

In [92]:
fig = px.bar(grouped_weekday_df, x=weekday_labels, y=weekday_values, 
            text=weekday_values, opacity=0.75)
fig.update_traces(texttemplate='%{text:.1f}', textposition='outside')
fig.update_layout(title_text='Average Number of Shooting Incidents Per Weekday', title_x=0.5)
fig.update_yaxes(visible=False)
fig.update_xaxes(title='', visible=True, showticklabels=True)
fig.show()

## In 2021, the level of gun violence has surpassed the results from 2020.  As of September 22, 2021, gun violence deaths have increased by 24.81% compared to the same period last year. 

In [93]:
start_date = '2020-01-01'
end_date = '2020-09-22'
mask = (shootings_df['date_']> start_date) & (shootings_df['date_'] <= end_date)

year_comparison_2020 = shootings_df.loc[mask]
year_comparison_2020.fatal.sum()

270.0

In [94]:
start_date2 = '2021-01-01'
end_date2 = '2021-09-22'
mask = (shootings_df['date_']> start_date2) & (shootings_df['date_'] <= end_date2)

year_comparison_2021 = shootings_df.loc[mask]
year_comparison_2021.fatal.sum()

337.0

In [95]:
Shooting_percentage_increase = (year_comparison_2021.fatal.sum() - year_comparison_2020.fatal.sum())/year_comparison_2020.fatal.sum()*100
Shooting_percentage_increase.round(2)


24.81

## Since 2015, the number of shooting incidents are higher on the weekends with approximately 229.5 incidents on Saturdays and 237.2 incidents on Sundays per year.

In [96]:
district_group = shootings_df.groupby('dist')['fatal'].sum().reset_index()
district_group = district_group.rename(columns={'dist': 'district', 'fatal': 'fatal shooting incident avg'})
district_group

,district,fatal shooting incident avg
0,1.0,25.0
1,2.0,44.0
2,3.0,29.0
3,5.0,5.0
4,6.0,19.0
5,7.0,7.0
6,8.0,18.0
7,9.0,9.0
8,12.0,170.0
9,14.0,126.0


In [97]:
district_group['% of fatal shootings'] = ((district_group['fatal shooting incident avg']/district_group['fatal shooting incident avg'].sum())*100).round(2)
district_group.sort_values(by='fatal shooting incident avg', ascending=False).head()

,district,fatal shooting incident avg,% of fatal shootings
17,25.0,247.0,11.88
15,22.0,234.0,11.26
16,24.0,205.0,9.86
20,39.0,173.0,8.32
8,12.0,170.0,8.18


## 5 Philadelphia police precint district areas account for 48.99% of all fatal shooting incidents.

In [98]:
district_df = shootings_df[shootings_df['dist'].isin([25.0, 22.0, 24.0, 12.0, 39.0]) & (shootings_df.fatal > 0.0)]

In [99]:
fatal_shootings_loc_dist = district_df[['lat', 'lng', 'dist', 'location']]
fatal_shootings_loc_dist = fatal_shootings_loc_dist.dropna()
fatal_shootings_loc_dist['dist'] = fatal_shootings_loc_dist['dist'].astype(str)

## The five most violent Philadelphia police precint areas are concentrated in North and West Philadelphia

In [100]:
fatal_shootings_map = folium.Map(location=[39.9509, -75.1575], 
                                 zoom_start=12, 
                                 zoom_control=False,
                                 scrollWheelZoom=False,
                                 dragging=False
)


for row in fatal_shootings_loc_dist.iterrows():
    row_values = row[1]
    location = [row_values['lat'], row_values['lng']]
    popup = popup = '<strong>' + row_values['location'] + '<strong>'
    marker = folium.Marker(location = location, popup = popup)
    marker.add_to(fatal_shootings_map)

display(fatal_shootings_map)

## Tragically, gun homocide victims are disproproportionality young, black men between the ages of 18 - 29

In [101]:
fatal_shootings_df = shootings_df[(shootings_df.fatal > 0.0)]

In [102]:
fig = px.histogram(fatal_shootings_df, x="age", template='simple_white+presentation', opacity=0.75)
fig.update_layout(title = "Gun Violence Victims Age Distribution", xaxis_title= "Ages", yaxis_title= "Count" )

fig.show()

In [103]:
fatal_shootings_df = shootings_df[(shootings_df.fatal > 0.0)]

victim_race = fatal_shootings_df["race"].value_counts()
victim_race_groups = victim_race.index.tolist()
victim_race_counts = victim_race.values.tolist()

victim_map = {'B':'Black', 'W':'White', 'A':'Unknown'}
victim_race_labels = [victim_map[x] for x in victim_race_groups]



In [104]:
fig = px.bar(victim_race, x=victim_race_labels, y=victim_race_counts, 
            text=victim_race_counts, opacity=0.75)
fig.update_layout(title = "Gun Violence Victims Race Distribution", xaxis_title= "Race", title_x=0.5)
fig.update_traces(texttemplate='%{text:,}', textposition='outside')
fig.update_yaxes(visible=False)
fig.show()

## Thank you for reviewing this analysis on Gun Violence in Philadelphia, please feel free to reach me at snellmatthewL@gmail.com, if you have any questions. 